# scratch

In [2]:
paths = ["/cis/home/tathey/projects/mouselight/sriram/somez_hier.zarr/" + str(i) for i in range(1,3)]
zarrs = [zarr.open(path) for path in paths]
slices = [z[0,:,:,:1] for z in zarrs]
print([z.shape for z in zarrs])
print([z.shape for z in slices])

[(2, 3407, 4224, 316), (2, 1703, 2112, 316)]
[(3407, 4224, 1), (1703, 2112, 1)]


In [47]:
napari.view_image(slices, multiscale=True, order=(2,0,1))

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1703.0, 2111.5), zoom=0.12408277076606984, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 3407.0, 1.0), (0.0, 4224.0, 1.0), (0.0, 1.0, 1.0)), current_step=(1703, 2112, 0), order=(2, 0, 1), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'slices' at 0x7f460d4ad820>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vert

Traceback (most recent call last):
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/app/backends/_qt.py", line 903, in paintGL
    self._vispy_canvas.events.draw(region=None)
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/scene/canvas.py", line 218, in on_draw
    self._draw_scene()
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/scene/canvas.py", line 277, in _draw_scene
    self.draw_visual(self.scene)
  File "/cis/home/tathey/projects/env_38/lib/python3.8

In [56]:
slices = [np.ones((int(100/2**i),int(100/2**i), 1)) for i in range(2)]
napari.view_image(slices, order=(2,0,1))

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 49.5, 49.5), zoom=10.241, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 100.0, 1.0), (0.0, 100.0, 1.0), (0.0, 1.0, 1.0)), current_step=(50, 50, 0), order=(2, 0, 1), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'slices' at 0x7f45fe62af70>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=False, selection_b

Traceback (most recent call last):
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/app/backends/_qt.py", line 903, in paintGL
    self._vispy_canvas.events.draw(region=None)
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/scene/canvas.py", line 218, in on_draw
    self._draw_scene()
  File "/cis/home/tathey/projects/env_38/lib/python3.8/site-packages/vispy/scene/canvas.py", line 277, in _draw_scene
    self.draw_visual(self.scene)
  File "/cis/home/tathey/projects/env_38/lib/python3.8

In [22]:
zim = zarr.open("/cis/home/tathey/projects/mouselight/sriram/somez_hier.zarr/1/")
print(zim.shape)
slice1 = zim[0,:,:,10]

(2, 3407, 4224, 316)


In [28]:
import zarr
import numpy as np
from cloudvolume import CloudVolume

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 3406.5, 4223.5), zoom=0.10972262987965953, angles=(0.0, 0.0, 90.0), perspective=0, interactive=True), cursor=Cursor(position=(1, 1), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 6814.0, 1.0), (0.0, 8448.0, 1.0)), current_step=(3407, 4224), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'pyramid' at 0x7f4498996a00>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=False, selection_box_drag=None, s

In [12]:
z = zarr.open(
    "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/3-1-soma.zarr"
)
ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/im"

info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="image",
    data_type="uint8",  # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, zfpc, compresso
    encoding="raw",
    resolution=[500, 500, 300],  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=z.shape,  # units are voxels
    volume_size=z.shape,  # e.g. a cubic millimeter dataset
)
vol = CloudVolume(ng_path, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = z[:, :, :]

In [15]:
z = zarr.open(
    "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/3-1-soma_labels.zarr"
)
ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/frags"

info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="segmentation",
    data_type="uint16",  # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, zfpc, compresso
    encoding="raw",
    resolution=[500, 500, 300],  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=z.shape,  # units are voxels
    volume_size=z.shape,  # e.g. a cubic millimeter dataset
)
vol = CloudVolume(ng_path, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = z[:, :, :]

Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.83it/s]
